In [ ]:
import pandas as pd
import requests
from datetime import datetime

def fetch_candles(symbol, interval, start_str, end_str=None):
    url = 'https://api.binance.com/api/v3/klines'
    data = []
    limit = 1000
    start_time = int(pd.to_datetime(start_str).timestamp() * 1000)
    end_time = int(pd.to_datetime(end_str).timestamp() * 1000) if end_str else None

    while True:
        params = {
            'symbol': symbol,
            'interval': interval,
            'startTime': start_time,
            'limit': limit
        }
        if end_time:
            params['endTime'] = end_time

        response = requests.get(url, params=params)
        new_data = response.json()
        if not new_data:
            break
        data.extend(new_data)
        start_time = new_data[-1][0] + 1

    columns = ['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time',
               'Quote asset volume', 'Number of trades', 'Taker buy base asset volume',
               'Taker buy quote asset volume', 'Ignore']
    df = pd.DataFrame(data, columns=columns)
    df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
    df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
    df.set_index('Open time', inplace=True)
    df = df[['Open', 'High', 'Low', 'Close']]
    return df

start_date = (datetime.now() - pd.DateOffset(years=4)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

btc_data = fetch_candles('ETHUSDT', '1d', start_date, end_date)


btc_data.to_csv('ETH_daily_candles2.csv')

print(btc_data.head())